## 6.3.1 padding

2-dim conv layer

In [20]:
import torch
import torch.nn as nn
import numpy as np

In [6]:
def pad_conv2d(pad, input, kernel):
    padded = torch.ones(input.shape[0]+pad*2, input.shape[1]+pad*2)
    padded[pad:input.shape[0]+pad, pad:input.shape[1]+pad] = input
    print(padded.shape)
    k_h, k_w = kernel.shape[0], kernel.shape[1]
    out_size = [padded.shape[0]-k_h+1, padded.shape[1]-k_w+1]
    output = torch.zeros(out_size, dtype=torch.float32)
    
    for i in range(out_size[0]):
        for j in range(out_size[1]):
            output[i, j] = (padded[i:i+k_h, j:j+k_w] * kernel).sum()
            
    return output

In [7]:
X = torch.ones(6,8)
X[:, 2:6] = 0
K = torch.Tensor([[1, -1]])
X, K, pad_conv2d(0, X, K)

torch.Size([6, 8])


(tensor([[1., 1., 0., 0., 0., 0., 1., 1.],
         [1., 1., 0., 0., 0., 0., 1., 1.],
         [1., 1., 0., 0., 0., 0., 1., 1.],
         [1., 1., 0., 0., 0., 0., 1., 1.],
         [1., 1., 0., 0., 0., 0., 1., 1.],
         [1., 1., 0., 0., 0., 0., 1., 1.]]),
 tensor([[ 1., -1.]]),
 tensor([[ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.],
         [ 0.,  1.,  0.,  0.,  0., -1.,  0.]]))

In [13]:
X = torch.rand(size=(8, 8))
K = torch.randn(3,3)
conv2d = nn.Conv2d(in_channels=1,out_channels=1,kernel_size=3,padding=1)
X, K, pad_conv2d(1, X, K), pad_conv2d(1, X, K).shape, conv2d(X.reshape(1,1, X.shape[0], X.shape[1])).shape

torch.Size([10, 10])
torch.Size([10, 10])


(tensor([[0.4524, 0.1933, 0.7932, 0.3182, 0.6474, 0.9099, 0.7452, 0.7475],
         [0.3564, 0.9284, 0.1694, 0.6654, 0.4113, 0.1532, 0.1088, 0.6975],
         [0.3212, 0.9623, 0.5122, 0.1980, 0.2479, 0.2753, 0.8209, 0.5198],
         [0.3846, 0.5112, 0.6789, 0.8449, 0.7111, 0.4067, 0.9715, 0.8780],
         [0.0583, 0.2051, 0.1125, 0.6189, 0.3062, 0.6472, 0.3071, 0.1311],
         [0.2598, 0.5291, 0.1472, 0.4688, 0.5587, 0.1194, 0.6290, 0.9241],
         [0.3525, 0.4608, 0.2989, 0.4459, 0.7447, 0.0444, 0.1484, 0.4844],
         [0.8553, 0.2102, 0.4567, 0.7591, 0.2605, 0.5160, 0.3691, 0.9907]]),
 tensor([[ 0.0304,  0.0919,  0.9487],
         [-0.3488,  0.4826,  0.3844],
         [-0.1661, -0.2884,  1.2448]]),
 tensor([[1.9017, 1.1955, 2.1340, 1.4887, 1.5839, 1.5938, 2.2121, 2.5819],
         [1.3747, 1.4796, 0.3333, 1.2542, 1.1961, 1.6838, 1.4372, 2.6818],
         [1.4795, 1.4400, 1.4338, 0.9975, 0.5206, 1.4657, 1.9216, 2.1953],
         [1.0787, 1.0289, 1.4422, 0.8973, 1.1094, 1.2780,

## 6.3.2 Stride

In [45]:
def pad_stride_conv2d(pad, stride, input, kernel):
    if not isinstance(pad, list):
        pad = [pad]*2
    if not isinstance(stride, list):
        stride = [stride]*2
        
    padded = torch.ones(input.shape[0]+pad[0]*2, input.shape[1]+pad[1]*2)
    padded[pad[0]:input.shape[0]+pad[0], pad[1]:input.shape[1]+pad[1]] = input
    print(padded.shape)
    k_h, k_w = kernel.shape[0], kernel.shape[1]
    out_size = [int((padded.shape[0]-k_h+stride[0])/stride[0]), int((padded.shape[1]-k_w+stride[1])/stride[1])]

    output = torch.zeros(out_size, dtype=torch.float32)

    for i in range(0,out_size[0]):
        for j in range(0,out_size[1]):
            h, w = i*stride[0], j*stride[1] 
            output[i, j] = (padded[h:h+k_h, w:w+k_w] * kernel).sum()

    return output

In [50]:
X = torch.rand(size=(8, 8))
K = torch.rand(3,3)
K2 = torch.rand(3,5)
pad_stride_conv2d(1, 2, X, K).shape, pad_stride_conv2d([0,1], [3,4], X, K2).shape

torch.Size([10, 10])
torch.Size([8, 10])


(torch.Size([4, 4]), torch.Size([2, 2]))

In [49]:
conv2d_ps = nn.Conv2d(1, 1, 3, padding=1, stride=2)
conv2d_ps2 = nn.Conv2d(in_channels=1,out_channels=1, kernel_size=(3, 5), padding=(0, 1), stride=(3, 4))
X = torch.rand(size=(1, 1, 8, 8))
X.shape, conv2d_ps(X).shape, conv2d_ps2(X).shape

(torch.Size([1, 1, 8, 8]), torch.Size([1, 1, 4, 4]), torch.Size([1, 1, 2, 2]))